<a href="https://colab.research.google.com/github/nhutit05/nhutit05/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import  Dataset, DataLoader

Input File

In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

In [ ]:
import os
import string

## Keep your training documents in a folder named 'data'
input_data_dir = r"data"

# String of punctuation without the full stop
punctuation = string.punctuation.replace('.', '')  # Retain the full stop

def is_hidden(filepath):
    return os.path.basename(filepath).startswith('.')

text_data=""

howManyDocuments = 50
used=0
if os.path.isdir(input_data_dir):
  for filename in os.listdir(input_data_dir):
      filepath = os.path.join(input_data_dir, filename)
      if not is_hidden(filepath):
          with open(filepath, "r", encoding="utf-8") as infile:
              for line in infile:
                  if line.strip():  # Check if line is not just whitespace
                      # Remove all punctuation except full stops
                      for char in punctuation:
                          line = line.replace(char, '')
                      text_data += line
      used=used+1
      if (used==howManyDocuments):
          print(howManyDocuments, "documents used.")
          break

50 documents used.


In [ ]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text_data.lower())

In [ ]:
words

['german',
 'industrial',
 'employment',
 'seen',
 'stagnating',
 'the',
 'number',
 'of',
 'workers',
 'employed',
 'in',
 'the',
 'west',
 'german',
 'industrial',
 'sector',
 'stagnated',
 'in',
 'the',
 'last',
 'quarter',
 'of',
 '1986',
 'as',
 'a',
 '50',
 '000',
 'increase',
 'in',
 'overall',
 'employment',
 'benefited',
 'only',
 'the',
 'services',
 'branch',
 'the',
 'diw',
 'economic',
 'institute',
 'said',
 'a',
 'diw',
 'report',
 'added',
 'the',
 'general',
 'downturn',
 'in',
 'the',
 'economy',
 'since',
 'last',
 'autumn',
 'had',
 'had',
 'a',
 'negative',
 'effect',
 'on',
 'the',
 'willingness',
 'of',
 'firms',
 'to',
 'take',
 'on',
 'workers',
 'it',
 'referred',
 'to',
 'a',
 'marked',
 'downturn',
 'in',
 'the',
 'number',
 'of',
 'workers',
 'taken',
 'on',
 'in',
 'the',
 'capital',
 'goods',
 'sector',
 'new',
 'orders',
 'for',
 'manufacturing',
 'industry',
 'goods',
 'have',
 'mostly',
 'fallen',
 'or',
 'stagnated',
 'in',
 'recent',
 'months',
 'but

In [ ]:
# Create a set of all unique words and create a dictionary to convert words to integers
word_to_int = {w: i for i, w in enumerate(set(words))}
int_to_word = {i: w for w, i in word_to_int.items()}


In [ ]:
len(word_to_int)

2317

In [ ]:
# Convert the entire text to integers
encoded_text = np.array([word_to_int[word] for word in words])

In [ ]:
# Prepare dataset
def create_sequences(input_data, seq_length):
    sequences = []
    for i in range(0, len(input_data) - seq_length):
        sequence_in = input_data[i:i + seq_length]
        sequence_out = input_data[i + seq_length]
        sequences.append((sequence_in, sequence_out))
    return sequences

In [ ]:
seq_length = 20  # Length of input sequences
sequences = create_sequences(encoded_text, seq_length)

In [ ]:
sequences

[(array([1521,  116, 1346,  949,  869,  746, 2314, 1893, 1342, 1450, 1601,
          746, 1074, 1521,  116,  919, 2082, 1601,  746,  296]),
  2063),
 (array([ 116, 1346,  949,  869,  746, 2314, 1893, 1342, 1450, 1601,  746,
         1074, 1521,  116,  919, 2082, 1601,  746,  296, 2063]),
  1893),
 (array([1346,  949,  869,  746, 2314, 1893, 1342, 1450, 1601,  746, 1074,
         1521,  116,  919, 2082, 1601,  746,  296, 2063, 1893]),
  1023),
 (array([ 949,  869,  746, 2314, 1893, 1342, 1450, 1601,  746, 1074, 1521,
          116,  919, 2082, 1601,  746,  296, 2063, 1893, 1023]),
  1458),
 (array([ 869,  746, 2314, 1893, 1342, 1450, 1601,  746, 1074, 1521,  116,
          919, 2082, 1601,  746,  296, 2063, 1893, 1023, 1458]),
  1813),
 (array([ 746, 2314, 1893, 1342, 1450, 1601,  746, 1074, 1521,  116,  919,
         2082, 1601,  746,  296, 2063, 1893, 1023, 1458, 1813]),
  2095),
 (array([2314, 1893, 1342, 1450, 1601,  746, 1074, 1521,  116,  919, 2082,
         1601,  746,  296, 2063

In [ ]:
class RNNModel(nn.Module):
    def __init__(self,
                 input_size,
                 output_size,
                 hidden_dim,
                 n_layers):
        super(RNNModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(
            input_size, hidden_dim)
        self.rnn = nn.RNN(
            hidden_dim, hidden_dim,
            n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden):
        x = self.embedding(x)
        out, hidden = self.rnn(x, hidden)
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(self.n_layers,
                           batch_size,
                           self.hidden_dim).to(device)

In [ ]:
# Check for GPU
device = torch.device(
    "cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
input_size = len(word_to_int)
output_size = len(word_to_int)
hidden_dim = 256
n_layers = 3
batch_size = 256
epochs = 90

# Initialize model, loss function, and optimizer
model = RNNModel(
    input_size,
    output_size,
    hidden_dim,
    n_layers).to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(
    model.parameters(), lr=0.001)

# Convert sequences to PyTorch tensors
import numpy as np

def batchify(data, batch_size):
    # Prepare inputs and targets
    inputs = np.array([item[0] for item in data])
    targets = np.array([item[1] for item in data])

    # Split data into batches
    n_batches = len(inputs) // batch_size
    inputs = inputs[:n_batches * batch_size]
    targets = targets[:n_batches * batch_size]

    # Batchify
    batched_inputs = [torch.tensor(inputs[i:i + batch_size], dtype=torch.long)
                      for i in range(0, len(inputs), batch_size)]
    batched_targets = [torch.tensor(targets[i:i + batch_size], dtype=torch.long)
                       for i in range(0, len(targets), batch_size)]

    return batched_inputs, batched_targets

batched_inputs, batched_targets = batchify(sequences, batch_size)


# Training loop
# Training loop
for epoch in range(epochs):
    for batch_idx in range(len(batched_inputs)):
        inputs = batched_inputs[batch_idx].to(device)
        targets = batched_targets[batch_idx].to(device)

        hidden = model.init_hidden(batch_size)
        optimizer.zero_grad()
        output, hidden = model(inputs, hidden)

        # Reshape output to [batch_size, seq_length, output_size]
        output = output.view(batch_size, seq_length, -1)

        # Use only the last output of each sequence
        output_last = output[:, -1, :]

        # Flatten the last outputs
        output_flat = output_last.view(-1, output_size)

        # Flatten the targets
        targets_flat = targets.view(-1)

        # Compute the loss
        loss = loss_function(output_flat, targets_flat)

        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')



Epoch 1, Loss: 7.320459842681885
Epoch 2, Loss: 7.026331424713135
Epoch 3, Loss: 6.8489670753479
Epoch 4, Loss: 6.645021915435791
Epoch 5, Loss: 6.446058750152588
Epoch 6, Loss: 6.172845840454102
Epoch 7, Loss: 5.906968593597412
Epoch 8, Loss: 5.616435527801514
Epoch 9, Loss: 5.3239288330078125
Epoch 10, Loss: 5.022075176239014
Epoch 11, Loss: 4.730508327484131
Epoch 12, Loss: 4.524765491485596
Epoch 13, Loss: 4.378107070922852
Epoch 14, Loss: 4.0505571365356445
Epoch 15, Loss: 3.691157579421997
Epoch 16, Loss: 3.3313534259796143
Epoch 17, Loss: 3.0797512531280518
Epoch 18, Loss: 2.8189799785614014
Epoch 19, Loss: 2.5480141639709473
Epoch 20, Loss: 2.3274500370025635
Epoch 21, Loss: 2.0091679096221924
Epoch 22, Loss: 1.7507556676864624
Epoch 23, Loss: 1.5658650398254395
Epoch 24, Loss: 1.3560664653778076
Epoch 25, Loss: 1.192384958267212
Epoch 26, Loss: 1.0455862283706665
Epoch 27, Loss: 0.8445541858673096
Epoch 28, Loss: 0.7132379412651062
Epoch 29, Loss: 0.604249894618988
Epoch 30, L

In [ ]:
import torch.nn.functional as F

start_seq = " Analysts noted that Bowater s profits"
model.eval()
input_seq = [word_to_int[word]
             for word in start_seq.lower().split()]
input_tensor = torch.tensor([input_seq],
                            dtype=torch.long).to(device)
hidden = model.init_hidden(1)
output, _ = model(input_tensor, hidden)
probabilities = F.softmax(output[-1], dim=0).detach().cpu()

# Choose from top k probabilities
top_prob, top_idx = torch.topk(probabilities, k=1)
next_word = int_to_word[top_idx.numpy()[0]]

In [ ]:
next_word

'of'

In [ ]:
import torch.nn.functional as F
import random

def generate_text(model,
                  start_seq,
                  word_to_int,
                  int_to_word,
                  gen_length=50,
                  top_k=1):
    """
    Generate text using a trained RNN model.

    Parameters:
    model: Trained RNN model.
    start_seq: Starting sequence for text generation.
    word_to_int: Dictionary mapping words to integers.
    int_to_word: Dictionary mapping integers to words.
    gen_length: Number of words to generate.
    top_k: from top_k next words, randomly select one.
    This ensures that results are different for the
    same prompt.

    Returns:
    Generated text.
    """
    model.eval()  # Set the model to evaluation mode

    input_seq = [word_to_int[word] for word in start_seq.lower().split()]
    generated_text = start_seq

    for _ in range(gen_length):
        input_tensor = torch.tensor([input_seq], dtype=torch.long).to(device)
        hidden = model.init_hidden(1)

        output, _ = model(input_tensor, hidden)
        probabilities = F.softmax(output[-1], dim=0).detach().cpu()

        # Choose from top k probabilities
        top_prob, top_idx = torch.topk(probabilities, k=top_k)
        chosen_idx = random.choice(top_idx.numpy())

        next_word = int_to_word[chosen_idx]
        generated_text += ' ' + next_word

        input_seq.append(chosen_idx)
        input_seq = input_seq[1:]

    return generated_text

def generate_text_temperature(model, start_seq,
                              word_to_int,
                              int_to_word,
                              gen_length=50,
                              temperature=1.0):
    # In this function, the temperature parameter
    # is used to scale the logits before applying
    # softmax. A higher temperature (>1) produces
    # more randomness, while a lower temperature (<1)
    # makes the model more confident (but potentially
    # more repetitive).

    model.eval()
    input_seq = [word_to_int.get(word, word_to_int['<unk>'])
                 for word in start_seq.lower().split()]
    generated_text = start_seq

    for _ in range(gen_length):
        input_tensor = torch.tensor([input_seq], dtype=torch.long).to(device)
        hidden = model.init_hidden(1)

        output, _ = model(input_tensor, hidden)
        output = output / temperature  # Adjusting output with temperature
        probabilities = F.softmax(output[-1], dim=0).detach().cpu()

        next_word_idx = torch.multinomial(probabilities, 1).item()
        next_word = int_to_word[next_word_idx]
        generated_text += ' ' + next_word

        input_seq.append(next_word_idx)
        input_seq = input_seq[1:]

    return generated_text

def beam_search(model,
                start_seq,
                word_to_int,
                int_to_word,
                beam_width=5,
                gen_length=50):
    # Initialize beams as a list of tuples (sequence, probability)
    input_seq = [word_to_int[word] for word in start_seq.lower().split()]
    beams = [(input_seq, 1.0)]

    for _ in range(gen_length):
        new_beams = []
        for seq, prob in beams:
            input_tensor = torch.tensor([seq], dtype=torch.long).to(device)
            hidden = model.init_hidden(1)

            output, _ = model(input_tensor, hidden)
            probabilities = F.softmax(output[-1], dim=0).detach().cpu().numpy()

            # Consider top beam_width choices for this beam
            top_indices = np.argsort(probabilities)[-beam_width:]
            for idx in top_indices:
                new_seq = seq + [idx]
                new_prob = prob * probabilities[idx]
                new_beams.append((new_seq, new_prob))

        # Select top beam_width beams
        beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_width]

    # Choose the beam with the highest probability
    best_seq, _ = max(beams, key=lambda x: x[1])
    return ' '.join([int_to_word[idx] for idx in best_seq])

In [30]:
start_sequence = "Analysts noted that Bowater "
generated_sequence_length = 100  # Number of words to generate

generated_text = beam_search(model,
                             start_sequence,
                             word_to_int, int_to_word,

                            gen_length=generated_sequence_length)

print(generated_text)

analysts noted that bowater s profits of 18 9 mln stg from 13 2 mln previously had been given a boost by pension benefits of 4 5 mln stg profit from australia and the far east showed the greatest percentage rise jumping 55 0 pct to 15 5 mln from 10 0 mln while the profit from u k operations rose 30 7 pct to 24 7 mln and europe 42 9 pct to 11 0 mln cra sold forrest gold for 76 mln dlrs whim creek lt whim creek consolidated nl said the consortium it is leading will pay 76 55 mln dlrs
